# Convert non encrypted ukbiobank data file to csv

Getting the converter and converting the unencrypted file to csv to read into dataframe

R plugin https://kenhanscombe.github.io/ukbtools/articles/explore-ukb-data.html

Converting exe https://biobank.ctsu.ox.ac.uk/crystal/download.cgi?id=101&ty=ut

Dataset usaage pdf https://biobank.ndph.ox.ac.uk/~bbdatan/Accessing_UKB_data_v2.3.pdf


In [ ]:
!wget  -nd  biobank.ctsu.ox.ac.uk/ukb/util/ukbconv

--2022-03-03 13:55:28--  http://biobank.ctsu.ox.ac.uk/ukb/util/ukbconv
Resolving biobank.ctsu.ox.ac.uk (biobank.ctsu.ox.ac.uk)... 163.1.206.99
Connecting to biobank.ctsu.ox.ac.uk (biobank.ctsu.ox.ac.uk)|163.1.206.99|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://biobank.ctsu.ox.ac.uk/ukb/util/ukbconv [following]
--2022-03-03 13:55:28--  https://biobank.ctsu.ox.ac.uk/ukb/util/ukbconv
Connecting to biobank.ctsu.ox.ac.uk (biobank.ctsu.ox.ac.uk)|163.1.206.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2000848 (1.9M)
Saving to: ‘ukbconv’

ukbconv             100%[===================>]   1.91M  3.47MB/s    in 0.5s    

2022-03-03 13:55:29 (3.47 MB/s) - ‘ukbconv’ saved [2000848/2000848]



In [ ]:
!chmod 755 ukbconv #make binary executable

In [ ]:
#convert to csv
!sudo ukbconv /content/ukb42668.enc_ukb csv

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# from tensorflow import keras

# BioBankData = pd.read_csv("/content/ukb42668.csv", encoding= 'unicode_escape', keep_default_na=False)#, nrows=26000) #UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 18283: invalid start byte
# print(BioBankData.shape)
# BioBankData.head()

Find rows where we have values for fundus images and save those rows

In [ ]:
#all image columns
# 21011-0.0, 21011-0.1, 21011-1.0, 21011-1.1, 21012-0.0, 21012-0.1, 21012-1.0, 21012-1.1, 21013-0.0, 21013-0.1, 21013-1.0, 21013-1.1, 21014-0.0, 21014-0.1, 21014-1.0, 21014-1.1, 21015-0.0, 21015-0.1, 21015-1.0, 21015-1.1, 21016-0.0	21016-0.1, 21016-1.0, 21016-1.1

#21016 for right eye fundus
#all fundus images
# 21015-0.0	21015-0.1	21015-1.0	21015-1.1	21016-0.0	21016-0.1	21016-1.0	21016-1.1
#left eye fundus images
# 21015-0.0, 21015-0.1, 21015-1.0, 21015-1.1

# 21015-0.0, 21015-1.0

#oct images on server: (OCT image slices (right))
#21018_0_0, 21018_1_0

chunksize = 25000#how many rows to process at a time, change depending on avalible ram
header = True#use header for first chunk
for chunk in pd.read_csv("/content/ukb42668.csv", chunksize=chunksize, encoding= 'unicode_escape'):#writing chunked dataframe
  #select rows for chunk with condition - not nan in wanted columns

  #for simple left eye fundus images
  # selected_rows = chunk[~chunk['21015-0.0'].isnull() | ~chunk['21015-1.0'].isnull() ]
  #for all fundus images
  selected_rows = chunk[~chunk['21015-0.0'].isnull() | ~chunk['21015-0.1'].isnull() | ~chunk['21015-1.0'].isnull() | ~chunk['21015-1.1'].isnull() | ~chunk['21016-0.0'].isnull() | ~chunk['21016-0.1'].isnull() | ~chunk['21016-1.0'].isnull() | ~chunk['21016-1.1'].isnull()]
  
  #save with append
  selected_rows.to_csv("/content/ukb42668_chunked.csv",  header=header, mode='a')#mode a is append - header only in first chunk
  header = False

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (29,31,382,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1995,2204,2206,2216,2218,2240,2242,3107,3108,3109,3110,3111,3112,3600,3601,3602,3605,3606,3607,3760,3761,3762,3763,5744,5745,6125,6126,6127,6129,6131,6133,6135,6136,6137,6379,6380,6381,6382,6383,6384,6385,6386,6387,6388,6582,6583,6584,6586,6587,6588,6590,6591,6592,6594,6595,6596,6598,6599,6600,6602,6603,6604,6629,7405,7406,7407,7408,7409,7410,7411,7412,9535,9536,9537,9538,9539,9540,9541,9542,9543,9544,9545,9546,9547,9548,9549,9550,9551,9552,9858,9859,9860,9861,9862,9863,9864,9865,9866,9867,9868,9891,9892,9893,9894,9895,9908,9909,9910,9911,9912,9944,10013,10014,10016,10017,10018,10019,10020,10033,10043,10044,10045,10046,10047,10048,10049,10050,10051,10052,10053,10054,10055,100

For those rows, remove any fully empty columns = not needed

Cant remove columns when taking chunks into account, will result in odd column shifts - need to do the whole thing

In [ ]:
# #replace empty spaces with nan and remove all fully empty columns
# nan_value = float("NaN")
# chunksize = 25000#how many rows to process at a time
# header = True#use header for first chunk
# for chunk in pd.read_csv("ukb42668_chunked.csv", chunksize=chunksize, encoding= 'unicode_escape'):#writing chunked dataframe
#   chunk.replace("", nan_value, inplace=True)#replace any empty strings with NAN
#   chunk.dropna(how='all', axis=1, inplace=True)#drop full na rows or columns
  
#   #save with append
#   chunk.to_csv("ukb42668_fundus_noemptycolumns.csv",  header=header, mode='a')#mode a is append - header only in first chunk
#   header = False

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (32,383,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1971,1972,1973,1974,1975,1976,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,2207,2219,2243,3110,3111,3112,3113,3603,3604,3606,3607,3608,3609,3610,3762,3764,5745,5746,5747,6126,6128,6130,6132,6134,6136,6137,6138,6380,6381,6382,6383,6384,6385,6386,6387,6388,6389,6583,6585,6587,6589,6591,6593,6595,6597,6599,6601,6603,6605,6630,7406,7407,7408,7409,7410,7411,7412,7413,7414,9536,9537,9538,9539,9540,9541,9542,9543,9544,9545,9546,9547,9548,9549,9550,9551,9552,9553,9859,9861,9863,9864,9865,9866,9867,9868,9869,9892,9893,9894,9895,9896,9897,9909,9910,9911,9912,9913,9914,9945,10014,10017,10018,10019,10020,10021,10022,10034,10043,10044,10045,10046,10047,10048,10049,10050,10051,10052,10053,10054,10055,10056,10057,10058,10059,10060,10061,10062,10063,10064,10065,10066,10067,10068,10069,10070,10071,10072,10073,100

Try to filter nans from the whole thing - high ram runtime reccomended

In [ ]:
import pandas as pd
nan_value = float("NaN")
path = "ukb42668_chunked.csv"#r'E:\BioBankData\ukb42668_chunked.csv'
outputPath = "ukb42668_chunked_emptyRemoved.csv" #r"E:\BioBankData\ukb42668_fundus_noemptycolumns.csv"


chunk= pd.read_csv(path) #writing chunked dataframe
chunk.replace("", nan_value, inplace=True)#replace any empty strings with NAN
chunk.dropna(how='all', axis=1, inplace=True)#drop full na rows or columns

#save with append
chunk.to_csv(outputPath, index = None, header=True)#mode a is append - header only in first chunk

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (32,383,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,2207,2219,2243,3108,3109,3110,3111,3112,3113,3603,3604,3606,3607,3608,3609,3610,3762,3764,5745,5746,5747,6126,6128,6130,6132,6134,6136,6137,6138,6380,6381,6382,6383,6384,6385,6386,6387,6388,6389,6583,6585,6587,6589,6591,6593,6595,6597,6599,6601,6603,6605,6630,7406,7407,7408,7409,7410,7411,7412,7413,7414,7415,7416,7417,7418,7419,9536,9537,9538,9539,9540,9541,9542,9543,9544,9545,9546,9547,9548,9549,9550,9551,9552,9553,9859,9861,9863,9864,9865,9866,9867,9868,9869,9870,9871,9872,9873,9892,9893,9894,9895,9896,9897,9909,9910,9911,9912,9913,9914,9945,10014,10017,10018,10019,10020,10021,10022,10034,10043,10044,10045,10046,10047,10048,10049,10050,10051,10052,10053,100

For our new dataset, read our images and find those that match our dataset, download images first

In [ ]:
import os
from os import listdir
from os.path import isfile, join
mypath = r"E:\Fundus images\images" #image folder location
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]#get the imagenames from folder

ids = []
for filename in onlyfiles:
    ids.append(filename.partition('_')[0])#get the ids from the images

ids =  [int(i) for i in ids]

import pandas as pd
chunksize = 25000#how many rows to process at a time
header = True#use header for first chunk
for chunk in pd.read_csv(r"D:\Users\giles\Downloads\ukb42668_fundus_noemptycolumns.csv", chunksize=chunksize, encoding= 'unicode_escape'):#writing chunked dataframe
  chunk = chunk.loc[chunk['eid'].isin(ids)]

  #save with append
  chunk.to_csv(r"D:\Users\giles\Downloads\ukb42668_fundus_noemptycolumns_validRows.csv",  header=header, mode='a')#mode a is append - header only in first chunk
  header = False

In [ ]:
while True:pass

#Download dataframe for biobank data

#Get the relevent patents (filter for diagnosis)

In [ ]:
#load as df
import pandas as pd

biobankdata = pd.read_csv("ukb42668_chunked_emptyRemoved_validRows.csv")
biobankdata = biobankdata.astype('string')
display(biobankdata)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (32,251,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1680,1681,1682,1683,1684,1685,1686,1687,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1854,1866,1890,2716,2717,2718,2719,3188,3189,3190,3191,3192,3193,3194,3331,3333,4615,4616,4617,4918,4920,4922,4924,4926,4928,4929,4930,5098,5099,5100,5101,5102,5103,5104,5105,5106,5107,5300,5302,5304,5306,5308,5310,5312,5314,5316,5318,5320,5322,5347,5876,5877,5878,5879,5880,5881,5882,5883,5884,5886,7767,7768,7769,7770,7771,7772,7773,7774,7775,7776,7777,7778,7779,7780,7781,7782,7783,7784,8090,8091,8092,8093,8094,8095,8096,8097,8098,8104,8105,8106,8107,8108,8109,8111,8112,8113,8114,8115,8116,8126,8155,8157,8158,8159,8160,8161,8162,8164,8172,8173,8174,8175,8176,8177,8178,8179,8180,8181,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191,8192,8193,8194,8195,8196,8197,8198,8199,8200,8201,8202,8203,8208,8209,8210,8211,82

,Unnamed: 0,eid,21-0.0,21-1.0,21-2.0,21-3.0,31-0.0,34-0.0,36-0.0,36-1.0,...,105010-0.0,105010-1.0,105010-2.0,105010-3.0,105010-4.0,105030-0.0,105030-1.0,105030-2.0,105030-3.0,105030-4.0
0,0,1000026,1.0,1.0,1.0,<NA>,1,1946,1051.0,3160.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1,1000098,1.0,<NA>,<NA>,<NA>,0,1966,1059.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,2012-05-30T09:10:45,<NA>,<NA>,<NA>,<NA>,2012-05-30T08:48:14
2,2,1000240,1.0,1.0,1.0,<NA>,0,1942,3780.0,3160.0,...,2009-11-03T16:08:24,<NA>,<NA>,<NA>,<NA>,2009-11-03T15:44:22,<NA>,<NA>,<NA>,<NA>
3,3,1000262,1.0,<NA>,<NA>,<NA>,1,1961,3155.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,4,1000327,1.0,1.0,<NA>,<NA>,1,1940,1051.0,1055.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21385,21395,2248224,1.0,1.0,<NA>,<NA>,0,1952,3155.0,3160.0,...,2009-08-05T16:17:03,2011-03-15T16:43:52,<NA>,2011-11-15T16:44:13,2012-05-20T12:40:38,2009-08-05T16:04:45,2011-03-15T16:29:43,<NA>,2011-11-15T16:31:03,2012-05-20T12:25:06
21386,21396,2248240,1.0,<NA>,<NA>,<NA>,0,1950,2816.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21387,21397,2248327,1.0,<NA>,1.0,1.0,0,1956,1041.0,<NA>,...,<NA>,2011-04-06T08:38:56,2011-08-15T22:25:40,2011-12-01T07:52:35,2012-06-13T09:04:52,<NA>,2011-04-06T08:30:34,2011-08-15T22:16:34,2011-12-01T07:46:52,2012-06-13T08:56:23
21388,21398,2248359,1.0,<NA>,<NA>,<NA>,1,1964,1042.0,<NA>,...,2010-06-07T17:25:57,<NA>,2011-06-26T12:30:56,2011-10-14T08:30:12,2012-04-17T12:25:11,2010-06-07T17:02:15,<NA>,2011-06-26T12:04:30,2011-10-14T08:12:12,2012-04-17T12:09:02


In [ ]:
#filter columns for eid, image, diagnosis
id_coluimns = ["eid"]

#left eye fundus images
# 21015-0.0, 21015-1.0
# fundus_columns = "21015-0.0 21015-1.0"
# fundus_columns = fundus_columns.split(" ")

#all fundus image columns
fundus_columns = '21015-0.0','21015-0.1','21015-1.0','21015-1.1','21016-0.0','21016-0.1','21016-1.0','21016-1.1'



#diagnosis columns
#for icd 10
# 41270-0.0 41270-0.1 41270-0.2 41270-0.3 41270-0.4 41270-0.5 41270-0.6 41270-0.7 41270-0.8 41270-0.9 41270-0.10 41270-0.11 41270-0.12 41270-0.13 41270-0.14 41270-0.15 41270-0.16 41270-0.17 41270-0.18 41270-0.19 41270-0.20 41270-0.21 41270-0.22 41270-0.23 41270-0.24 41270-0.25 41270-0.26 41270-0.27 41270-0.28 41270-0.29 41270-0.30 41270-0.31 41270-0.32 41270-0.33 41270-0.34 41270-0.35 41270-0.36 41270-0.37 41270-0.38 41270-0.39 41270-0.40 41270-0.41 41270-0.42 41270-0.43 41270-0.44 41270-0.45 41270-0.46 41270-0.47 41270-0.48 41270-0.49 41270-0.50 41270-0.51 41270-0.52 41270-0.53 41270-0.54 41270-0.55 41270-0.56 41270-0.57 41270-0.58 41270-0.59 41270-0.60 41270-0.61 41270-0.62 41270-0.63 41270-0.64 41270-0.65 41270-0.66 41270-0.67 41270-0.68 41270-0.69 41270-0.70 41270-0.71 41270-0.72 41270-0.73 41270-0.74 41270-0.75 41270-0.76 41270-0.77 41270-0.78 41270-0.79 41270-0.80 41270-0.81 41270-0.82 41270-0.83 41270-0.84 41270-0.85 41270-0.86 41270-0.87 41270-0.88 41270-0.89 41270-0.90 41270-0.91 41270-0.92 41270-0.93 41270-0.94 41270-0.95 41270-0.96 41270-0.97 41270-0.98 41270-0.99 41270-0.100 41270-0.101 41270-0.102 41270-0.103 41270-0.104 41270-0.105 41270-0.106 41270-0.107 41270-0.108 41270-0.109 41270-0.110 41270-0.111 41270-0.112 41270-0.113 41270-0.114 41270-0.115 41270-0.116 41270-0.117 41270-0.118 41270-0.119 41270-0.120 41270-0.121 41270-0.122 41270-0.123 41270-0.124 41270-0.125 41270-0.126 41270-0.127 41270-0.128 41270-0.129 41270-0.130 41270-0.131 41270-0.132 41270-0.133 41270-0.134 41270-0.135 41270-0.136 41270-0.137 41270-0.138 41270-0.139 41270-0.140 41270-0.141 41270-0.142 41270-0.143 41270-0.144 41270-0.145 41270-0.146 41270-0.147 41270-0.148 41270-0.149 41270-0.150 41270-0.151 41270-0.152 41270-0.153 41270-0.154 41270-0.155 41270-0.156 41270-0.157 41270-0.158 41270-0.159 41270-0.160 41270-0.161 41270-0.162 41270-0.163 41270-0.164 41270-0.165 41270-0.166 41270-0.167 41270-0.168 41270-0.169 41270-0.170 41270-0.171 41270-0.172 41270-0.173 41270-0.174 41270-0.175 41270-0.176 41270-0.177 41270-0.178 41270-0.179 41270-0.180 41270-0.181 41270-0.182 41270-0.183 41270-0.184 41270-0.185 41270-0.186 41270-0.187 41270-0.188 41270-0.189 41270-0.190 41270-0.191 41270-0.192 41270-0.193 41270-0.194 41270-0.195 41270-0.196 41270-0.197 41270-0.198 41270-0.199 41270-0.200 41270-0.201
diagnosis_columns = "41270-0.0 41270-0.1 41270-0.2 41270-0.3 41270-0.4 41270-0.5 41270-0.6 41270-0.7 41270-0.8 41270-0.9 41270-0.10 41270-0.11 41270-0.12 41270-0.13 41270-0.14 41270-0.15 41270-0.16 41270-0.17 41270-0.18 41270-0.19 41270-0.20 41270-0.21 41270-0.22 41270-0.23 41270-0.24 41270-0.25 41270-0.26 41270-0.27 41270-0.28 41270-0.29 41270-0.30 41270-0.31 41270-0.32 41270-0.33 41270-0.34 41270-0.35 41270-0.36 41270-0.37 41270-0.38 41270-0.39 41270-0.40 41270-0.41 41270-0.42 41270-0.43 41270-0.44 41270-0.45 41270-0.46 41270-0.47 41270-0.48 41270-0.49 41270-0.50 41270-0.51 41270-0.52 41270-0.53 41270-0.54 41270-0.55 41270-0.56 41270-0.57 41270-0.58 41270-0.59 41270-0.60 41270-0.61 41270-0.62 41270-0.63 41270-0.64 41270-0.65 41270-0.66 41270-0.67 41270-0.68 41270-0.69 41270-0.70 41270-0.71 41270-0.72 41270-0.73 41270-0.74 41270-0.75 41270-0.76 41270-0.77 41270-0.78 41270-0.79 41270-0.80 41270-0.81 41270-0.82 41270-0.83 41270-0.84 41270-0.85 41270-0.86 41270-0.87 41270-0.88 41270-0.89 41270-0.90 41270-0.91 41270-0.92 41270-0.93 41270-0.94 41270-0.95 41270-0.96 41270-0.97 41270-0.98 41270-0.99 41270-0.100 41270-0.101 41270-0.102 41270-0.103 41270-0.104 41270-0.105 41270-0.106 41270-0.107 41270-0.108 41270-0.109 41270-0.110 41270-0.111 41270-0.112 41270-0.113 41270-0.114 41270-0.115 41270-0.116 41270-0.117 41270-0.118 41270-0.119 41270-0.120 41270-0.121 41270-0.122 41270-0.123 41270-0.124 41270-0.125 41270-0.126 41270-0.127 41270-0.128 41270-0.129 41270-0.130 41270-0.131 41270-0.132 41270-0.133 41270-0.134 41270-0.135 41270-0.136 41270-0.137 41270-0.138 41270-0.139 41270-0.140 41270-0.141 41270-0.142 41270-0.143 41270-0.144 41270-0.145 41270-0.146 41270-0.147 41270-0.148 41270-0.149 41270-0.150 41270-0.151 41270-0.152 41270-0.153 41270-0.154 41270-0.155 41270-0.156 41270-0.157 41270-0.158 41270-0.159 41270-0.160 41270-0.161 41270-0.162 41270-0.163 41270-0.164 41270-0.165 41270-0.166 41270-0.167 41270-0.168 41270-0.169 41270-0.170 41270-0.171 41270-0.172 41270-0.173 41270-0.174 41270-0.175 41270-0.176 41270-0.177 41270-0.178 41270-0.179 41270-0.180 41270-0.181 41270-0.182 41270-0.183 41270-0.184 41270-0.185 41270-0.186 41270-0.187 41270-0.188 41270-0.189 41270-0.190 41270-0.191 41270-0.192 41270-0.193 41270-0.194 41270-0.195 41270-0.196 41270-0.197 41270-0.198 41270-0.199 41270-0.200 41270-0.201"
diagnosis_columns = diagnosis_columns.split(" ")

needed_columns = id_coluimns, fundus_columns, diagnosis_columns
needed_columns = [item for sublist in needed_columns for item in sublist]

diagnosis_crossref =  biobankdata[needed_columns]
display(diagnosis_crossref)

,eid,21015-0.0,21015-0.1,21015-1.0,21015-1.1,21016-0.0,21016-0.1,21016-1.0,21016-1.1,41270-0.0,...,41270-0.192,41270-0.193,41270-0.194,41270-0.195,41270-0.196,41270-0.197,41270-0.198,41270-0.199,41270-0.200,41270-0.201
0,1000026,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,H259,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1000098,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,C509,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,1000240,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,B980,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1000262,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,F171,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1000327,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21385,2248224,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,C509,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21386,2248240,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,I10,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21387,2248327,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,K30,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21388,2248359,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
diagnosis_crossref.to_csv("ukb42668_fundus_noemptycolumns_ALLEYE_validRows_diagnosis.csv", index = None, header=True)

#Get specific diagnosis/no diagnosis'

In [ ]:
#look for wanted diagnosis
import numpy as np

#for pure samples = no diagnosis
pure_samples=diagnosis_crossref[diagnosis_crossref[diagnosis_columns].isnull().all(1)]#pure sample, no diagnosis in ICD10
contain_values = pure_samples#pure sample

#for specific diagnosis
#for Alzhimers in ICD 10 general
diagnosis = "M2534"#"G30"

# mask = np.column_stack([diagnosis_crossref[col].str.contains(diagnosis, na=False) for col in diagnosis_crossref])
# contain_values = diagnosis_crossref.loc[mask.any(axis=1)]
display(contain_values)

#for rows with it take the image name
imageNamesList = []
patientEIDs = []

contain_values = contain_values.fillna("NONE")
for index, row in contain_values.iterrows():
  #row can have multiple images
  # '21015-0.0','21015-0.1','21015-1.0','21015-1.1','21016-0.0','21016-0.1','21016-1.0','21016-1.1'
  patientEIDs.append(row["eid"])
  imagesForPatient = []

  if row["21015-0.0"] is not "NONE":
    imagename = "21015_0_0"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21015-0.1"] is not "NONE":
    imagename = "21015_0_1"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21015-1.0"] is not "NONE":
    imagename = "21015_1_0"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21015-1.1"] is not "NONE":
    imagename = "21015_1_1"
    imagesForPatient.append(row["eid"]+"_"+imagename)


  if row["21016-0.0"] is not "NONE":
    imagename = "21016_0_0"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21016-0.1"] is not "NONE":
    imagename = "21016_0_1"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21016-1.0"] is not "NONE":
    imagename = "21016_1_0"
    imagesForPatient.append(row["eid"]+"_"+imagename)

  if row["21016-1.1"] is not "NONE":
    imagename = "21016_1_1"
    imagesForPatient.append(row["eid"]+"_"+imagename)
  
  imageNamesList.append(imagesForPatient)


  
#add image name to file
imageNamesList = [[name + ".png" for name in sublist] for sublist in imageNamesList]#want to make each element of sublists have .png for each ele
print(imageNamesList)
print(set(patientEIDs))#print unique patients
instanceNumber = sum(len(row) for row in imageNamesList)
print(f"number of images: {instanceNumber}")
print(f"number of patents: {len(set(patientEIDs))}")

intermediate_dictionary = {'Patent eid':patientEIDs, 'imageFileName':imageNamesList}

# Convert dictionary to Pandas dataframe
pandas_dataframe = pd.DataFrame(intermediate_dictionary)

display(pandas_dataframe)
pandas_dataframe.to_csv("pureSampleICD10.csv", index = None, header=True)

,eid,21015-0.0,21015-0.1,21015-1.0,21015-1.1,21016-0.0,21016-0.1,21016-1.0,21016-1.1,41270-0.0,...,41270-0.192,41270-0.193,41270-0.194,41270-0.195,41270-0.196,41270-0.197,41270-0.198,41270-0.199,41270-0.200,41270-0.201
4,1000327,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
53,1002861,21015_0_0,<NA>,21015_1_0,<NA>,21016_0_0,<NA>,21016_1_0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
62,1003396,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,21016_0_1,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
75,1004064,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
82,1004376,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21371,2247419,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21378,2247773,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21379,2247858,21015_0_0,<NA>,<NA>,<NA>,21016_0_0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21381,2247934,<NA>,<NA>,21015_1_0,<NA>,<NA>,<NA>,21016_1_0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


[['1000327_21015_1_0.png', '1000327_21016_1_0.png'], ['1002861_21015_0_0.png', '1002861_21015_1_0.png', '1002861_21016_0_0.png', '1002861_21016_1_0.png'], ['1003396_21015_0_0.png', '1003396_21016_0_0.png', '1003396_21016_0_1.png'], ['1004064_21015_0_0.png', '1004064_21016_0_0.png'], ['1004376_21015_0_0.png', '1004376_21016_0_0.png'], ['1004393_21015_0_0.png', '1004393_21016_0_0.png'], ['1004642_21015_0_0.png', '1004642_21016_0_0.png'], ['1005107_21015_0_0.png', '1005107_21016_0_0.png'], ['1005124_21015_1_0.png', '1005124_21016_1_0.png'], ['1005239_21015_0_0.png', '1005239_21016_0_0.png'], ['1005544_21015_1_0.png', '1005544_21016_1_0.png'], ['1005648_21015_1_0.png', '1005648_21016_1_0.png'], ['1006498_21015_0_0.png', '1006498_21016_0_0.png'], ['1007050_21015_0_0.png', '1007050_21016_0_0.png'], ['1007106_21015_0_0.png', '1007106_21016_0_0.png'], ['1007129_21015_0_0.png', '1007129_21016_0_0.png'], ['1007612_21015_0_0.png', '1007612_21016_0_0.png'], ['1008150_21015_0_0.png', '1008150_21016

,Patent eid,imageFileName
0,1000327,"[1000327_21015_1_0.png, 1000327_21016_1_0.png]"
1,1002861,"[1002861_21015_0_0.png, 1002861_21015_1_0.png,..."
2,1003396,"[1003396_21015_0_0.png, 1003396_21016_0_0.png,..."
3,1004064,"[1004064_21015_0_0.png, 1004064_21016_0_0.png]"
4,1004376,"[1004376_21015_0_0.png, 1004376_21016_0_0.png]"
...,...,...
4199,2247419,"[2247419_21015_0_0.png, 2247419_21016_0_0.png]"
4200,2247773,"[2247773_21015_1_0.png, 2247773_21016_1_0.png]"
4201,2247858,"[2247858_21015_0_0.png, 2247858_21016_0_0.png]"
4202,2247934,"[2247934_21015_1_0.png, 2247934_21016_1_0.png]"


#Get all diagnosis specific patent ids and images

In [ ]:
#load ICD10 list
#download the list here: https://biobank.ndph.ox.ac.uk/ukb/coding.cgi?id=19&nl=1
#YOU NEED TO MOVE THE TOP PARENT FOLDERS TO THE TOP OF THE FILE AS THESE NEED TO BE WORKED OUT LAST
#These sections start with "Chapter I" and "Block A00-A09"
#and Block M30-M36 set to non selectable
diagnosisIndex = pd.read_csv("coding19.csv")#, sep='\t')
diagnosisIndex = diagnosisIndex.astype('string')
display(diagnosisIndex)

In [ ]:
!pip install tqdm
# import modin.pandas as pd
# import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
#def generic method
diagnosis_crossrefWithoutNA = diagnosis_crossref.fillna("NONE")
def find_all_samples(diagnosisString):
  contain_values = diagnosis_crossrefWithoutNA[diagnosis_crossrefWithoutNA.eq(diagnosisString).any(axis = 1)]#find rows with any columns having the diagnosis
  #for rows with it take the image name
  imageNamesList = []
  patientEIDs = []#for patent number - can be gained from image name though
  for index, row in contain_values.iterrows():
    #row can have multiple images
    # '21015-0.0','21015-0.1','21015-1.0','21015-1.1','21016-0.0','21016-0.1','21016-1.0','21016-1.1'
    patientEIDs.append(row["eid"])
    if row["21015-0.0"] is not "NONE":
      imagename = "21015_0_0"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21015-0.1"] is not "NONE":
      imagename = "21015_0_1"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21015-1.0"] is not "NONE":
      imagename = "21015_1_0"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21015-1.1"] is not "NONE":
      imagename = "21015_1_1"
      imageNamesList.append(row["eid"]+"_"+imagename)

    if row["21016-0.0"] is not "NONE":
      imagename = "21016_0_0"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21016-0.1"] is not "NONE":
      imagename = "21016_0_1"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21016-1.0"] is not "NONE":
      imagename = "21016_1_0"
      imageNamesList.append(row["eid"]+"_"+imagename)
    if row["21016-1.1"] is not "NONE":
      imagename = "21016_1_1"
      imageNamesList.append(row["eid"]+"_"+imagename)

  imageNamesList = [name + ".png" for name in imageNamesList]
  return (len(imageNamesList), imageNamesList, len(patientEIDs), patientEIDs)


# patients_length = []
# image_name_list = []

# nonSelectRows = diagnosisIndex.reset_index().loc[diagnosisIndex["selectable"] == "N"]#get all non selectable rows
# nonSelectRows['NonSelectNext'] = nonSelectRows['index'].shift(-1)#get the next non selectable rows index

diagnosisIndexWithCounts = diagnosisIndex.copy()

diagnosisIndexWithCounts["numberOfPatients"] = ""#create empty columns
diagnosisIndexWithCounts["PatientIds"] = ""

diagnosisIndexWithCounts["numberOfImages"] = ""#create empty columns
diagnosisIndexWithCounts["fundusImageNames"] = ""


display(diagnosisIndexWithCounts)

#for every endpoint get the singular diagnosis
#might be posssible to parrellize
# display(diagnosisIndex.loc[::-1].loc[diagnosisIndex["selectable"]=='Y'])
for index, row in tqdm(diagnosisIndex.loc[::-1].loc[diagnosisIndex["selectable"] =='Y'].iterrows(), total=diagnosisIndex.loc[diagnosisIndex["selectable"] =='Y'].shape[0]):#itterate backwards first
  imgListSize, imageList, patientIDSize, patientIDs = find_all_samples(row["coding"])
  diagnosisIndexWithCounts.at[index, 'numberOfPatients'] = patientIDSize
  diagnosisIndexWithCounts.at[index, 'fundusImageNames'] = list(set(imageList))

  diagnosisIndexWithCounts.at[index, 'numberOfImages'] = imgListSize
  diagnosisIndexWithCounts.at[index, 'PatientIds'] = list(set(patientIDs))

#save incomplete as the first 287 items are folders that arent distributed top down like the others so cant be done sequentially
diagnosisIndexWithCounts.to_csv("diagnosisTreeIncomplete.csv", index = None, header=True)

#for the remaining folders/nodes - find children and collect values
diagnosisIndexWithCountsSlice = diagnosisIndexWithCounts#[:286]#overrite first set of Ns as it isnt distributed
for index, row in tqdm(diagnosisIndexWithCountsSlice.loc[::-1].loc[diagnosisIndex["selectable"] =='N'].iterrows(), total=diagnosisIndexWithCountsSlice.loc[diagnosisIndex["selectable"] =='N'].shape[0]):#go backwards
  # if row["selectable"] is "N":
    parentID = row["node_id"]#get the id of parent
    children = diagnosisIndexWithCounts.loc[diagnosisIndexWithCounts['parent_id'] == parentID]#get all rows where its the parent
    # totalNumPatients = children['numberOfPatients'].sum()#sum all the patents of those rows #I think these are inaccurate
    # totalNumIMG = children['numberOfImages'].sum()#sum all the patents of those rows

    listofIDsLists = children['PatientIds'].tolist()#get all the image names lists
    IDSList = [item for sublist in listofIDsLists for item in sublist]#flatten
    IDSList = list(set(IDSList))#remove duplicates

    listofImageLists = children['fundusImageNames'].tolist()#get all the image names lists
    imageList = [item for sublist in listofImageLists for item in sublist]#flatten
    imageList = list(set(imageList))#remove duplicates
    
    totalNumPatients=len(IDSList)
    totalNumIMG=len(imageList)

    diagnosisIndexWithCounts.at[index, 'numberOfPatients'] = totalNumPatients
    diagnosisIndexWithCounts.at[index, 'fundusImageNames'] = imageList

    diagnosisIndexWithCounts.at[index, 'numberOfImages'] = totalNumIMG
    diagnosisIndexWithCounts.at[index, 'PatientIds'] = IDSList



# #save to csv
diagnosisIndexWithCounts.to_csv("diagnosisTree.csv", index = None, header=True)

In [ ]:
# #save to csv with just counts
diagnosisIndexWithCountsNOIDS = diagnosisIndexWithCounts.copy()
diagnosisIndexWithCountsNOIDS.drop(['PatientIds', 'fundusImageNames'], axis = 1, inplace = True) 
diagnosisIndexWithCountsNOIDS.to_csv("diagnosisTreeNOIDS.csv", index = None, header=True)

#Get random negative sample of same size (not pure samples)

In [ ]:
print(patientEIDs)
doesNotHave = diagnosis_crossref[~diagnosis_crossref["eid"].isin(patientEIDs)]
display(doesNotHave)

doesNotHaveSample = doesNotHave.sample(instanceNumber)

NegativeSampleImageList = []
doesNotHaveSample = doesNotHaveSample.fillna("NONE")
for index, row in doesNotHaveSample.iterrows():
  if row["21015-1.0"] is "NONE":
    imagename = "21015_0_0"
  else:
    imagename = "21015_1_0"
  
  NegativeSampleImageList.append(row["eid"]+"_"+imagename)


NegativeSampleImageList = [name + ".png" for name in NegativeSampleImageList]
print(NegativeSampleImageList)

In [ ]:
# import the necessary packages
import numpy as np
import cv2
# import glob
import os
from tqdm import tqdm
import math
from PIL import Image
# files = glob.glob('D:\\Experiments with Deep Learning\\DR Kaggle\\train\\train\\train\\*.jpeg')

new_sz = 1024

def crop_image(image):
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print('no contours!')
        flag = 0
        return image, flag
    cnt = max(contours, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(cnt)
    x = int(x); y = int(y); r = int(r)
    flag = 1
    #print(x,y,r)
    if r > 100:
        return output[0 + (y-r)*int(r<y):-1 + (y+r+1)*int(r<y),0 + (x-r)*int(r<x):-1 + (x+r+1)*int(r<x)], flag
    else:
        print('none!')
        flag = 0
        return image,flag

In [ ]:
combinedLists = imageNamesList + NegativeSampleImageList
print(combinedLists)

#Utils

In [ ]:
#remove empty columns from whole dataframe
import pandas as pd

path = r'E:\BioBankData\ukb42668_chunked.csv'
chunk= pd.read_csv(path) #writing chunked dataframe
chunk.replace("", nan_value, inplace=True)#replace any empty strings with NAN
chunk.dropna(how='all', axis=1, inplace=True)#drop full na rows or columns

#save with append
chunk.to_csv(r"E:\BioBankData\ukb42668_fundus_noemptycolumns.csv", index = None, header=True)#mode a is append - header only in first chunk


In [ ]:
#Image cropping
%pip install opencv-python

# import the necessary packages
import numpy as np
import cv2
import glob
import os
from tqdm import tqdm
import math
from PIL import Image

files = glob.glob(r'E:\Fundus images\images\*.png')
NewDest = f"E:\\Fundus images\\CroppedIMG\\"

new_sz = 1024

def crop_image(image):
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print('no contours!')
        flag = 0
        return image, flag
    cnt = max(contours, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(cnt)
    x = int(x); y = int(y); r = int(r)
    flag = 1
    #print(x,y,r)
    if r > 100:
        return output[0 + (y-r)*int(r<y):-1 + (y+r+1)*int(r<y),0 + (x-r)*int(r<x):-1 + (x+r+1)*int(r<x)], flag
    else:
        print('none!')
        flag = 0
        return image,flag
#-------------------------------------------------------------------------------------------
for i in tqdm(range(len(files))):
    img = cv2.imread(files[i])
    img_cropped, flag = crop_image(img)
    if not flag:
        print(files[i])
    height, width, _= img_cropped.shape
    ratio = height/width
    if width > new_sz:
        new_image = cv2.resize(img_cropped,(new_sz,math.ceil(new_sz*ratio)))  
    else:
        new_image = img_cropped

    cv2.imwrite(NewDest + os.path.basename(files[i]), new_image)

In [ ]:
#From patient ids copy images to folder 
import pandas as pd

path = r'E:\BioBankData\diagnosisTree (1).csv'
chunk= pd.read_csv(path)

row=chunk.loc[chunk['coding'] == "Block N17-N19"]#find row with coding we want
display(row.index)

fundusImageNames = chunk.loc[row.index]['fundusImageNames']

import ast
ImageList = ast.literal_eval(fundusImageNames.values[0])#from string list create list
print("ImageListLen:",len(ImageList))
# print(ImageList)

PatientIds = chunk.iloc[row.index]['PatientIds'].values[0]
print("patientids String Len:",len(PatientIds))
# print(PatientIds)
PatientList = ast.literal_eval(PatientIds)
print("Patient ids list len:",len(PatientList))

from shutil import copyfile
from glob import glob
import os

for id in PatientList:
    print(id)
    # print(glob(f'E:\\Fundus images\\images\\{id}*'))
    for file in glob(f'E:\\Fundus images\\images\\{id}*'):#for any images matching the patient id
        dest = r'E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\{}'.format(os.path.basename(file))
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")

In [ ]:
#create folder of pure sample images, using patient ids
import pandas as pd

path = f'E:\BioBankData\pureSampleICD10 (1).csv'
pureSamplesDF= pd.read_csv(path)
display(pureSamplesDF)
ListOFIDS = pureSamplesDF['Patent eid'].tolist()
# print(ListOFIDS)

from shutil import copyfile
from glob import glob
import os

for id in ListOFIDS:
    print(id)
    for file in glob(f'E:\\Fundus images\\images\\{id}*'):
        dest = r'E:\Fundus images\PureSamples\{}'.format(os.path.basename(file))
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")

In [ ]:
#for using the hand cleaned data to auto partially clean larger sets
from shutil import copyfile
from glob import glob
import os

#original
orig = glob(r'E:\Fundus images\I10-I15_Hypertensive_diseases\original\\*')
orig = [os.path.basename(file) for file in orig]
cleaned = glob(r'E:\Fundus images\I10-I15_Hypertensive_diseases\cleaned\\*')
cleaned = [os.path.basename(file) for file in cleaned]

diff = list(set(orig) - set(cleaned))
print(len(diff))
# print(diff)


orig2 = glob(r'E:\Fundus images\N17-N19_Renal_failure\original\\*')
orig2 = [os.path.basename(file) for file in orig2]
cleaned2 = glob(r'E:\Fundus images\N17-N19_Renal_failure\cleaned\\*')
cleaned2 = [os.path.basename(file) for file in cleaned2]
diff2 = list(set(orig2) - set(cleaned2))
print(len(diff2))


orig3 = glob(r'E:\Fundus images\I80-I89_Diseases_of_veins_lymphatic_vessels_and_nodes\original\\*')
orig3 = [os.path.basename(file) for file in orig3]
cleaned3 = glob(r'E:\Fundus images\I80-I89_Diseases_of_veins_lymphatic_vessels_and_nodes\cleaned\\*')
cleaned3 = [os.path.basename(file) for file in cleaned3]
diff3 = list(set(orig3) - set(cleaned3))
print(len(diff3))

orig4 = glob(r'E:\Fundus images\I60-I69_Cerebrovascular_diseases\original\\*')
orig4 = [os.path.basename(file) for file in orig4]
cleaned4 = glob(r'E:\Fundus images\I60-I69_Cerebrovascular_diseases\cleaned\\*')
cleaned4 = [os.path.basename(file) for file in cleaned4]
diff4 = list(set(orig4) - set(cleaned4))
print(len(diff4))

orig5 = glob(r'E:\Fundus images\G30-G32_Other degenerative_diseases_of_nervous system\original\\*')
orig5 = [os.path.basename(file) for file in orig5]
cleaned5 = glob(r'E:\Fundus images\G30-G32_Other degenerative_diseases_of_nervous system\cleaned\\*')
cleaned5 = [os.path.basename(file) for file in cleaned5]
diff5 = list(set(orig5) - set(cleaned5))
print(len(diff5))

orig6 = glob(r'E:\Fundus images\Ch_VI_DiseasesOfNervousSystem\original\\*')
orig6 = [os.path.basename(file) for file in orig6]
cleaned6 = glob(r'E:\Fundus images\Ch_VI_DiseasesOfNervousSystem\cleaned\\*')
cleaned6 = [os.path.basename(file) for file in cleaned6]
diff6 = list(set(orig6) - set(cleaned6))
print(len(diff6))


badFiles =list(diff + diff2 + diff3 + diff4 + diff5 + diff6)
print(len(badFiles))
badFiles = set(badFiles)
print(len(badFiles))

for fileName in diff:
    try:
        os.remove(f"E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\cleaned\\{fileName}")
        print("deleted:",fileName)
    except:
        print("Error while deleting file ", fileName)

In [ ]:
#OIA-ODIR (https://github.com/nkicsl/OIA-ODIR)
# Ocular Disease Intelligent Recognition (ODIR-2019) dataset

#We are interested in some of the images to add to our dataset.
#for normal fundus images without other conditions (no disease)
#and for images with other conditions that we are studying.

#Normal
#Hypotention (H)

#do for each set, training, on/off site test

import pandas as pd
df = pd.read_excel(r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\Annotation\off-site test annotation (English).xlsx')
display(df)

#go down to the excel file and find the images that have the label "H"
#and save them to a new folder
H = df.loc[df['H'] == 1]
display(H)

#go through H and check eye columns to see which images are left and right
#save them to a new folder
for index, row in H.iterrows():
    if "hypertensive retinopathy" in row["Left-Diagnostic Keywords"]:
        name= row["Left-Fundus"]
        file = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\Images\\' + row["Left-Fundus"]
        dest = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\hypertension\{}'.format(name)
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")
    if "hypertensive retinopathy" in row["Right-Diagnostic Keywords"]:
        name= row["Right-Fundus"]
        file = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\Images\\' + row["Right-Fundus"]
        dest = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\hypertension\{}'.format(name)
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")
        
#Normal (N), is 1 when both are normal
#Will need to check each row

#go through H and check eye columns to see which images are left and right
#save them to a new folder
for index, row in df.iterrows():
    if "normal fundus" in row["Left-Diagnostic Keywords"]:
        name= row["Left-Fundus"]
        file = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\Images\\' + row["Left-Fundus"]
        dest = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\normal\{}'.format(name)
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")
    if "normal fundus" in row["Right-Diagnostic Keywords"]:
        name= row["Right-Fundus"]
        file = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\Images\\' + row["Right-Fundus"]
        dest = r'E:\Fundus images\Ocular Disease Intelligent Recognition (ODIR-2019)\Off-site Test Set\normal\{}'.format(name)
        copyfile(file, dest)
        print(f"copied: {file} to {dest}")

In [ ]:
#move cleaned & cropped positive samples into train directory as well as pure samples as comparison
import numpy as np
import cv2
from shutil import copyfile
from glob import glob
import os
import random
random.seed(0)
from tqdm import tqdm
import math
from PIL import Image

# move positive samples to train
PositiveSamples = glob(r'E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\cropped\\*')
for file in PositiveSamples:
    dest = r'E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\train\1\\' + os.path.basename(file)
    copyfile(file, dest)
    print(f"copied: {file} to {dest}")


#move pure samples to 0 folder in train with same size as 1
PositiveSamples = glob(r'E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\train\1\\*')
size = len(PositiveSamples)
print(size)

#load negative samples
NegativeSamples = glob(r'E:\Fundus images\PureSamples\croppedCombined\\*')
size2 = len(NegativeSamples)
print(size2)

# if(size < size2):
    #randmoize negative samples
random.shuffle(NegativeSamples)
for i in range(size):
    name = NegativeSamples[i]
    dest = r'E:\Fundus images\Ch_IX_Diseases_of_Circulatory_System\train\0\\{}'.format(os.path.basename(name))
    copyfile(name, dest)
    print(f"copied: {name} to {dest}")
#in the case where its greater, we already cleaned the rest and if we didnt the best we could get would be around 10k instead of 9.5k
#so not a massive improvement to include bad samples that were removed.
